In [6]:
import networkx as nx
import random
import numpy as np
import math

In [6]:
def modify_graph(graph):
    """
        Removes nodes and edges with degree 1
    """

    to_remove = []
    for node in graph.nodes():
        if graph.degree(node) == 1:
            to_remove.append(node)
    
    for node in to_remove:
        graph.remove_node(node)

# graph is the social network
graph = nx.Graph()

"""
# Constructing the graph
graph.add_node(1)
graph.add_node(2)
graph.add_node(3)

p12 = 0.65
p13 = 0.55
graph.add_edge(1,2, weight=p12)
graph.add_edge(1,3, weight=p13)
"""

graph = modify_graph(graph)

In [3]:
def random_position_function(graph):
    list_X = []
    max_degree = 0

    for node in graph.nodes():
        current_degree = graph.degree(node)
        max_degree = max(max_degree, current_degree)

        # Chance of a node being selected is proportional to its degree
        list_X.append(random.randint(1, current_degree))
    
    X = np.array(list_X)

    # Normalizing the chance of each node
    X = X / max_degree
    return X

def get_seed_set(graph, X, k):
    # This list will store the pair (probabilty of activation, node no.)
    max_list = []

    # All nodes in the graph
    list_of_nodes = list(graph.nodes)

    for i in range(len(list_of_nodes)):
        max_list.append((X[i], list_of_nodes[i]))

    max_list.sort(reverse=True)

    seed_set = [x for _, x in max_list]

    # returning k nodes with the highest probability of activation
    return seed_set[:k]

def fitness_func(graph, X, k):
    """
    Args:
        graph - Weighted graph where weights are
        the activation probabilty (Cascading Model)

        X - Position of the wolf. It is an array that represents the
        chance of the current node to be selected in the seed set

        k - The desired size of the seed set
    """

    dim = len(graph.nodes())

    seed_set = get_seed_set(graph, X, k)
    seed_set = set(seed_set)

    probability_first = dict()
    first_order_neighbour = set()

    probability_second = dict()
    second_order_neighbour = set()

    # Running BFS for a single iteration to get first order neighbours
    for node in seed_set:
        for neighbour in graph[node]:
            if neighbour not in seed_set:
                if neighbour in probability_first:
                    probability_first[neighbour] += graph[node][neighbour]['weight']
                else:
                    probability_first[neighbour] = graph[node][neighbour]['weight']
                
                first_order_neighbour.add(neighbour)

    # Running BFS for a single iteration to get second order neighbours
    for node in first_order_neighbour:
        for neighbour in graph[node]:
            if neighbour not in seed_set:
                if neighbour in probability_second:
                    probability_second[neighbour] += graph[node][neighbour]['weight'] * probability_first[node]
                else:
                    probability_second[neighbour] = graph[node][neighbour]['weight'] * probability_first[node]
                
                second_order_neighbour.add(neighbour)
    
    activation_probability = dict()

    # Adding contribution of first order neighbours
    activation_probability = probability_first.copy()

    # Adding contribution of second order neighbours
    for node in second_order_neighbour:
        if node in activation_probability:
            activation_probability[node] += probability_second[node]
        else:
            activation_probability[node] = probability_second[node]

    worthiness = dict()
    for node in activation_probability.keys():
        worthiness[node] = activation_probability[node] * graph.degree(node)

    # Total worthiness is sum of all worthiness
    W = sum(worthiness.values())

    fitness = 0

    # Calculating fitness using formula
    for w in worthiness.values():
        nw = w / W
        fitness += -(nw * math.log(nw))   # math.log(x) -> ln(x)
    
    return fitness


In [4]:
def GWIM(graph, k=10, pop=50, epoch=100):
    """
    Args:
        graph - Weighted graph where weights are the activation probabilty (Cascading Model)
        k - number of population size, default = 10
        epoch - maximum number of iterations, default = 100
    """

    # number of nodes in the graph
    dim = len(graph.nodes())

    # Population of wolves or the possible solutions
    wolves = []

    # Used to find alpha, beta & delta wolves
    max_list = []

    # Generating the population
    for i in range(pop):
        current_wolf = random_position_function(graph)
        current_fitness = fitness_func(graph, current_wolf, k)

        max_list.append((current_fitness, i))
        wolves.append(current_wolf)

    wolves = np.array(wolves)
    max_list.sort(reverse=True)

    # For finding aplha, beta, delta wolves
    alpha_index = max_list[0][1]
    beta_index = max_list[1][1]
    delta_index = max_list[2][1]
    
    alpha_fitness = max_list[0][0]
    beta_fitness = max_list[1][0]
    delta_fitness = max_list[2][0]

    alpha_wolf = wolves[alpha_index].copy()
    beta_wolf = wolves[beta_index].copy()
    delta_wolf = wolves[delta_index].copy()

    # Running the epochs
    for t in range(epoch):
        a = 2 - 2 * (t + 1) / epoch

        # Applying updation formula
        for i in range(pop):
            A1 = a * (2 * np.random.rand(dim) - 1)
            A2 = a * (2 * np.random.rand(dim) - 1)
            A3 = a * (2 * np.random.rand(dim) - 1)

            C1 = 2 * np.random.rand(dim)
            C2 = 2 * np.random.rand(dim)
            C3 = 2 * np.random.rand(dim)

            Y1 = alpha_wolf - A1 * np.abs(C1 * alpha_wolf - wolves[i])
            Y2 = beta_wolf - A2 * np.abs(C2 * beta_wolf - wolves[i])
            Y3 = delta_wolf - A3 * np.abs(C3 * delta_wolf - wolves[i])

            X = (Y1 + Y2 + Y3) / 3.0
            wolves[i] = X

            current_fitness = fitness_func(graph, X, k)

            # Updating positions of the wolves
            if current_fitness > alpha_fitness:
                alpha_fitness = current_fitness
                alpha_wolf = X.copy()
            elif current_fitness > beta_fitness:
                beta_fitness = current_fitness
                beta_wolf = X.copy()
            elif current_fitness > delta_fitness:
                delta_fitness = current_fitness
                delta_wolf = X.copy()

        # print(alpha_fitness)
    
    return get_seed_set(graph, alpha_wolf, k)

GWIM(graph, k=2)

[3, 2]